#                                           Reddit API and Webscape 

## The goals of the follwing code is:
### to import data from Reddit using API Calls
### Scrape Data using Selenium
### format collected strings and compile data into easy to follow lists and then make a master list (list of lists)
### create dataframe from master list  and save datataframe to a csv file for future data anaylsis

### 1. Reddit API Code

In [1]:
import praw  #Importing necessary modules
import pandas as pd
import datetime as dt
import reddit_cred
#reddit cred is a personal python script with my reddit info

In [2]:
from reddit_cred import client_id
from reddit_cred import client_secret #all of these credentials are needed to
from reddit_cred import user_agent    #get reddit data access
from reddit_cred import username
from reddit_cred import password

In [3]:
reddit = praw.Reddit(client_id=client_id, 
                     client_secret=client_secret, #praw is the method from Reddit
                     user_agent=user_agent, 
                     username=username, 
                     password=password)

In [4]:
subreddit = reddit.subreddit('moviesboxoffice') #movies box office is a subreddit I found
                                                #that contains table of weekly box office
                                                #sales data
                                                #will be vaulable for its budget numbers
                                                #gross domestic and worldwide sales numbers

In [5]:
new_subreddit = subreddit.new(limit=500)
#will select the most recent entries up to 500

In [6]:
dir(new_subreddit) #checking the methods and functions releted to 'new_subreddit'

['__abstractmethods__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__next__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_exhausted',
 '_is_protocol',
 '_list_index',
 '_listing',
 '_next_batch',
 '_reddit',
 '_safely_add_arguments',
 'limit',
 'params',
 'parse',
 'url',
 'yielded']

In [7]:
urls=[]; #For me the url method will be useful,as I can then collect the urls associated 
for i in new_subreddit:                                    #with the box office data
    urls.append(i.url)
len(urls)    
#Let's see how many urls we collected

202

In [8]:
urls[0] #the first url we collected

'https://www.reddit.com/r/movies/comments/bwag87/box_office_week_godzilla_king_of_the_monsters/'

### 2. Webscraping data using selenium library

In [9]:
from selenium import webdriver #Now, I import the selenium library and other necessary
import pandas as pd            #libraries for the data scraping
import numpy as np
import time
time.sleep(np.random.randint(0,4))

In [10]:
driver = webdriver.Chrome('/Users/scogs/Downloads/chromedriver')
#To use selenium,you need a 'driver' aka web browser that will automatically do what 
#commands you code                                            

In [11]:
driver.get(urls[0]) #I test this with the first url we collected

In [12]:
test=driver.find_element_by_class_name('MRH-njmSb5ZTkfb1o4dqv') 
#test is making sure I am scraping the correct table data
#the value in the string comes from inspecting the  table element on chrome and finding
#the class value

In [13]:
A=test.text #Now we take the text and I call it A
lists=[];
lists.append(A) #  append A to a blank list 'lists'
lists
#example test of what the scrape will look like

['Rank Title Domestic Gross (Weekend) Worldwide Gross (Cume) Week # Percentage Change Budget\n1 Godzilla: King of the Monsters $49,025,000 $179,025,000 1 N/A $170M\n2 Aladdin (2019) $42,335,000 $445,932,174 2 -53.7% $183M\n3 Rocketman $25,000,000 $56,200,000 1 N/A $40M\n4 Ma $18,260,000 $21,060,000 1 N/A $5M\n5 John Wick: Chapter 3 - Parabellum $11,100,000 $221,652,812 3 -54.9% $55M']

#### 2a. Scraping the Data with Selenium using for loop

In [14]:
from selenium.common.exceptions import NoSuchElementException #getting some strange errors
from selenium.common.exceptions import StaleElementReferenceException #so to correct I used
lists=[];                                                             #try-except solution
new_urls=urls[1:12]+urls[13:18]+urls[19:30]+urls[30:43]+urls[44:50]
#new_urls is a subset of urls because only this set contains budget data
# which we value highly
for url in new_urls:
    driver.get(url)
    try:
        test=driver.find_element_by_class_name('MRH-njmSb5ZTkfb1o4dqv')
    except (StaleElementReferenceException,StaleNameError,NoSuchElementException):
        continue
    finally:
        lists.append(test.text)
len(lists) 
#Compiled a list of all the information on all of the tables on the url set

46

In [15]:
lists[6] #what each table looks like as a list entry

'Rank Title Domestic Gross (Weekend) Worldwide Gross (Cume) Week # Percentage Change Budget\n1 Shazam! $25,140,000 $221,213,013 2 -53.0% $100M\n2 Little $15,499,000 $17,399,000 1 N/A $20M\n3 Hellboy (2019) $12,015,000 $12,015,000 1 N/A $50M\n4 Pet Sematary (2019) $10,000,000 $76,821,660 2 -59.2% $21M\n5 Dumbo (2019) $9,186,000 $266,945,162 3 -49.6% $170M'

In [16]:
urls[49] [''] #last url we scraped from

'https://www.reddit.com/r/movies/comments/8s03ji/box_office_week_incredibles_2_smashes_the/'

#### 3. String formatting and Data Organization

#### 3a. splitting data using end of line 

In [17]:
lists[6].split('\n') #seperating data by row

['Rank Title Domestic Gross (Weekend) Worldwide Gross (Cume) Week # Percentage Change Budget',
 '1 Shazam! $25,140,000 $221,213,013 2 -53.0% $100M',
 '2 Little $15,499,000 $17,399,000 1 N/A $20M',
 '3 Hellboy (2019) $12,015,000 $12,015,000 1 N/A $50M',
 '4 Pet Sematary (2019) $10,000,000 $76,821,660 2 -59.2% $21M',
 '5 Dumbo (2019) $9,186,000 $266,945,162 3 -49.6% $170M']

In [18]:
f_list=lists[6].split('\n') #testing to make sure the data is being seperated as desired

In [19]:
f_list[1] #what each row looks like

'1 Shazam! $25,140,000 $221,213,013 2 -53.0% $100M'

In [20]:
formatted_list=[]
for list in lists:
    formatted_list.append(list.split('\n'))
len(formatted_list) #looped through all of tables

46

In [21]:
formatted_list[6][5] #how a row from a table looks like now

'5 Dumbo (2019) $9,186,000 $266,945,162 3 -49.6% $170M'

#### 3b. Organizing data using string method from above

In [22]:
row_1=[]
row_2=[]
row_3=[]
row_4=[]
row_5=[]
row_6=[]
for i in range(len(formatted_list)):
    row_1.append(formatted_list[i][0])
    row_2.append(formatted_list[i][1])
    row_3.append(formatted_list[i][2])
    row_4.append(formatted_list[i][3])
    row_5.append(formatted_list[i][4])
    row_6.append(formatted_list[i][5])
print(len(row_6)) #seperated each entry by its row number

46


In [23]:
row_6[0].split('$') #testing how to further seperate distinct info

['5 Brightburn ', '9,500,000 ', '17,300,000 1 N/A ', '6M']

In [24]:
row_6[1].split('$') #2nd test to confirm 

['5 The Hustle ', '6,080,788 ', '51,245,512 2 -53.3% ', '43M']

In [25]:
row_1 #row 1 is column title so it is constant.so we only need it once.

['Rank Title Domestic Gross (Weekend) Worldwide Gross (Cume) Week # Percentage Change Budget',
 'Rank Title Domestic Gross (Weekend) Worldwide Gross (Cume) Week # Percentage Change Budget',
 'Rank Title Domestic Gross (Weekend) Worldwide Gross (Cume) Week # Percentage Change Budget',
 'Rank Title Domestic Gross (Weekend) Worldwide Gross (Cume) Week # Percentage Change Budget',
 'Rank Title Domestic Gross (Weekend) Worldwide Gross (Cume) Week # Percentage Change Budget',
 'Rank Title Domestic Gross (Weekend) Worldwide Gross (Cume) Week # Percentage Change Budget',
 'Rank Title Domestic Gross (Weekend) Worldwide Gross (Cume) Week # Percentage Change Budget',
 'Rank Title Domestic Gross (Weekend) Worldwide Gross (Cume) Week # Percentage Change Budget',
 'Rank Title Domestic Gross (Weekend) Worldwide Gross (Cume) Week # Percentage Change Budget',
 'Rank Title Domestic Gross (Weekend) Worldwide Gross (Cume) Week # Percentage Change Budget',
 'Rank Title Domestic Gross (Weekend) Worldwide Gr

In [26]:
r_2=[];
r_3=[];
r_4=[];
r_5=[];
r_6=[];
for i in range(len(row_2)):
    r_2.append(row_2[i].split('$'))
    r_3.append(row_3[i].split('$'))
    r_4.append(row_4[i].split('$'))
    r_5.append(row_5[i].split('$'))
    r_6.append(row_6[i].split('$'))
len(r_2) #applying the $ split to all of the rows and making sure length of row 2 is right

46

In [27]:
r_2 #what each row list looks like now

[['1 Aladdin (2019) ', '112,700,000 ', '233,700,000 1 N/A ', '183M'],
 ['1 John Wick: Chapter 3 - Parabellum ',
  '57,025,000 ',
  '149,225,000 1 N/A ',
  '55M'],
 ['1 Avengers: Endgame ', '63,054,000 ', '2,485,499,739 3 -57.2% ', '356M'],
 ['1 Avengers: Endgame ', '145,804,000 ', '2,188,698,638 2 -59.2% ', '356M'],
 ['1 Avengers: Endgame ', '350,000,000 ', '1,209,000,000 1 N/A ', '356M'],
 ['1 The Curse of La Llorona ', '26,505,000 ', '56,505,000 1 N/A ', '9M'],
 ['1 Shazam! ', '25,140,000 ', '221,213,013 2 -53.0% ', '100M'],
 ['1 Shazam! ', '53,450,000 ', '158,775,000 1 N/A ', '100M'],
 ['1 Dumbo (2019) ', '45,000,000 ', '116,000,000 1 N/A ', '150M'],
 ['1 Us ', '70,250,000 ', '86,950,000 1 N/A ', '20M'],
 ['1 Captain Marvel ', '69,318,000 ', '760,213,933 2 -54.8% ', '152M'],
 ['1 How to Train Your Dragon: The Hidden World ',
  '30,046,000 ',
  '375,396,275 2 -45.4% ',
  '129M'],
 ['1 How to Train Your Dragon: The Hidden World ',
  '55,527,000 ',
  '274,927,000 1 N/A ',
  '129M'],
 [

In [28]:
row_1 #row_1 is just the header from the scraped tables

['Rank Title Domestic Gross (Weekend) Worldwide Gross (Cume) Week # Percentage Change Budget',
 'Rank Title Domestic Gross (Weekend) Worldwide Gross (Cume) Week # Percentage Change Budget',
 'Rank Title Domestic Gross (Weekend) Worldwide Gross (Cume) Week # Percentage Change Budget',
 'Rank Title Domestic Gross (Weekend) Worldwide Gross (Cume) Week # Percentage Change Budget',
 'Rank Title Domestic Gross (Weekend) Worldwide Gross (Cume) Week # Percentage Change Budget',
 'Rank Title Domestic Gross (Weekend) Worldwide Gross (Cume) Week # Percentage Change Budget',
 'Rank Title Domestic Gross (Weekend) Worldwide Gross (Cume) Week # Percentage Change Budget',
 'Rank Title Domestic Gross (Weekend) Worldwide Gross (Cume) Week # Percentage Change Budget',
 'Rank Title Domestic Gross (Weekend) Worldwide Gross (Cume) Week # Percentage Change Budget',
 'Rank Title Domestic Gross (Weekend) Worldwide Gross (Cume) Week # Percentage Change Budget',
 'Rank Title Domestic Gross (Weekend) Worldwide Gr

#### 3c. Using scraped data to make column names for data frame

In [29]:
r_1=row_1[0].split(' ') #chose index 0 arbitrarily since row_1 is constant
r_1
cols=[]
cols.append(r_1[1])
cols.append(r_1[2]+" " +r_1[3] +' '+r_1[4])
cols.append(r_1[5]+" " +r_1[6] +' '+r_1[7])
cols.append(r_1[-5]+' '+r_1[-4])
cols.append(r_1[-1])
cols #making column list for dataframe

['Title',
 'Domestic Gross (Weekend)',
 'Worldwide Gross (Cume)',
 'Week #',
 'Budget']

In [30]:
r_1 #what row_1 looks like after formatting

['Rank',
 'Title',
 'Domestic',
 'Gross',
 '(Weekend)',
 'Worldwide',
 'Gross',
 '(Cume)',
 'Week',
 '#',
 'Percentage',
 'Change',
 'Budget']

In [31]:
r_2[0][2].split(' ') #testing how to seperate num of weeks 

['233,700,000', '1', 'N/A', '']

In [32]:
r_2[1][2].split(' ')[1] #now will use this formula in a loop

'1'

#### 3d. Collecting Number of Weeks since Release using for loop

In [33]:
week_num_2=[]
week_num_3=[]
week_num_4=[]
week_num_5=[]
for i in range(len(r_2)):
    week_num_2.append(r_2[i][2].split(' ')[1])
    week_num_3.append(r_3[i][2].split(' ')[1])
    week_num_4.append(r_4[i][2].split(' ')[1])
    week_num_5.append(r_5[i][2].split(' ')[1])
len(week_num_2) #making sure that list length is accurate 

46

In [34]:
week_num_3 #checking to make sure loop was performed correctly

['2',
 '4',
 '1',
 '1',
 '8',
 '3',
 '1',
 '1',
 '2',
 '3',
 '1',
 '1',
 '2',
 '2',
 '1',
 '4',
 '2',
 '4',
 '1',
 '2',
 '1',
 '1',
 '4',
 '1',
 '2',
 '2',
 '1',
 '4',
 '3',
 '2',
 '1',
 '1',
 '2',
 '1',
 '4',
 '3',
 '2',
 '3',
 '1',
 '2',
 '1',
 '2',
 '4',
 '3',
 '2',
 '2']

In [35]:
len(r_6)  #had some issues with length of row 6 so this was from a testign phase

46

In [36]:
num_weeks_6=[]
r_6[17][2]='N/A N/A N/A ' #was all blank when scraped
for i in range(len(r_6)):#thus loop was not working before so the above line is to fix
  num_weeks_6.append(r_6[i][2].split(' ')[1])



In [37]:
print(len(num_weeks_6),len(week_num_2)) #making sure lists'length is accurate

46 46


In [38]:
print(cols) #checking column list accuracy 
print(len(r_2),len(r_3),len(r_4),len(r_5),len(r_6)) #making sure row lists length is good
r_2[3][-1].split(' ')[0] #experimenting  on how to exclude budget info

['Title', 'Domestic Gross (Weekend)', 'Worldwide Gross (Cume)', 'Week #', 'Budget']
46 46 46 46 46


'356M'

#### 3e. Collecting distinct data into large lists 

In [39]:
#Now to make all desired info into seperate lists,row by row (each cell is a new row)
#try on row 2 elements
Title=[]
Domestic=[]
World=[]
Budget=[]
for i in range(len(r_2)):
    Title.append(r_2[i][0][2:].strip(' '))
    Domestic.append(r_2[i][1].strip(' '))
    World.append(r_2[i][2].split(' ')[0])
    Budget.append(r_2[i][-1].split(' ')[0])
#checking to make sure eveything matches    
#Budget[-1] is an exception so I'll fix it
Budget[-1]=np.nan
print(Title[-1],World[-1],Budget[-1],r_2[-1])#making sure it is fixed

Incredibles 2 231,500,000 nan ['1 Incredibles 2 ', '180,000,000 ', '231,500,000 1 N/A UNK']


In [40]:
#add row 3
for i in range(len(r_3)):
    Title.append(r_3[i][0][2:].strip(' '))
    Domestic.append(r_3[i][1].strip(' '))
    World.append(r_3[i][2].split(' ')[0])
    Budget.append(r_3[i][-1].split(' ')[0])

In [41]:
#add row 4
for i in range(len(r_4)):
    Title.append(r_4[i][0][2:].strip(' '))
    Domestic.append(r_4[i][1].strip(' '))
    World.append(r_4[i][2].split(' ')[0])
    Budget.append(r_4[i][-1].split(' ')[0])

In [42]:
#add row 5
for i in range(len(r_5)):
    Title.append(r_5[i][0][2:].strip(' '))
    Domestic.append(r_5[i][1].strip(' '))
    World.append(r_5[i][2].split(' ')[0])
    Budget.append(r_5[i][-1].split(' ')[0])

In [43]:
#add row 6
for i in range(len(r_6)):
    Title.append(r_4[i][0][2:].strip(' '))
    Domestic.append(r_4[i][1].strip(' '))
    World.append(r_4[i][2].split(' ')[0])
    Budget.append(r_4[i][-1].split(' ')[0])
print(len(Title),len(Domestic),len(World),len(Budget))

230 230 230 230


In [44]:
#need to add number of weeks column
number_of_weeks=[]
number_of_weeks.extend(week_num_2)
number_of_weeks.extend(week_num_3)
number_of_weeks.extend(week_num_4)
number_of_weeks.extend(week_num_5)
number_of_weeks.extend(num_weeks_6)


In [45]:
len(number_of_weeks)
#Matches other list's length, all good

230

In [46]:
cols[3]='Number of Weeks since Release' #making number of weeks column title easier to understand
cols

['Title',
 'Domestic Gross (Weekend)',
 'Worldwide Gross (Cume)',
 'Number of Weeks since Release',
 'Budget']

#### 3f. collecting all data into a master list which is a list of lists

In [47]:
#Time to put lists together and make dataframe
Master_list=[]
for i in range(len(Title)): #Title chosen arbitrarily, all lists have same length
    Master_list.append((Title[i],Domestic[i],World[i],number_of_weeks[i],Budget[i]))
len(Master_list) #should be 230

230

In [48]:
Master_list[-1] #Example of a movie from Master_list as a tuple

('Tag', '14,600,000', '16,000,000', '5', '28M')

#### 4. DataFrame Creation

In [49]:
#Making Dataframe
Reddit_data=pd.DataFrame(Master_list)
Reddit_data.columns=cols #changing columns names to desired
Reddit_data.head() #checking first 5 rows of new Dataframe

,Title,Domestic Gross (Weekend),Worldwide Gross (Cume),Number of Weeks since Release,Budget
0,Aladdin (2019),"112,700,000","233,700,000",1,183M
1,John Wick: Chapter 3 - Parabellum,"57,025,000","149,225,000",1,55M
2,Avengers: Endgame,"63,054,000","2,485,499,739",3,356M
3,Avengers: Endgame,"145,804,000","2,188,698,638",2,356M
4,Avengers: Endgame,"350,000,000","1,209,000,000",1,356M


In [50]:
Reddit_data.shape #shape is accurate (230 rows and 5 columns)

(230, 5)

#### 4a. Saving Dataframe for future Analysis

In [51]:
#All looks good, time to save dataframe to csv.temporaily save to desktop
Reddit_data=Reddit_data.to_csv(r'/Users/scogs/Desktop/Reddit_data.csv',index=False,header=True)